In [11]:
!pip install opencv-contrib-python
!pip install tensorflow

In [2]:
!pip uninstall opencv-contrib-python

^C


In [ ]:
import cv2

In [47]:
import os
from skimage import transform
from skimage import exposure
import argparse
import cv2
import imutils
from skimage import io
import numpy as np
import random
import matplotlib.pyplot as plt
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers

In [38]:
def load_split(basePath, csvPath):

    data = []
    labels = []
    rows = open(csvPath).read().strip().split("\n")[1:]
    random.shuffle(rows)
    for (i, row) in enumerate(rows):
        (label, imagePath) = row.strip().split(",")[-2:]
        imagePath = os.path.sep.join([basePath, imagePath])
        image = io.imread(imagePath)
        image = transform.resize(image, (32, 32))
        image = exposure.equalize_adapthist(image, clip_limit=0.1)
           
        data.append(image)
        labels.append(int(label))

    data = np.array(data)
    labels = np.array(labels)
    return (data, labels)


In [39]:
Names=['Speed limit (20km/h)','Speed limit (30km/h)','Speed limit (50km/h)','Speed limit (60km/h)','Speed limit (70km/h)',
'Speed limit (80km/h)','End of speed limit (80km/h)','Speed limit (100km/h)','Speed limit (120km/h)','No passing','No passing veh over 3.5 tons',
'Right-of-way at intersection','Priority road','Yield','Stop','No vehicles','Veh > 3.5 tons prohibited','No entry','General caution',
'Dangerous curve left','Dangerous curve right','Double curve','Bumpy road','Slippery road','Road narrows on the right','Road work', 
'Traffic signals','Pedestrians','Children crossing','Bicycles crossing','Beware of ice/snow','Wild animals crossing','End speed + passing limits', 
'Turn right ahead','Turn left ahead','Ahead only','Go straight or right','Go straight or left','Keep right','Keep left','Roundabout mandatory', 
'End of no passing','End no passing veh > 3.5 tons']

In [69]:
def plot_sample(X, y, index):

    plt.imshow(X[index])
    plt.xlabel(Names[y[index]])

In [41]:
trainPath = os.path.sep.join(["D:/Course/Deep Learning/archive", "Train.csv"])
testPath = os.path.sep.join(["D:/Course/Deep Learning/archive", "Test.csv"])
(trainX, trainY) = load_split("D:/Course/Deep Learning/archive", trainPath)
(testX, testY) = load_split("D:/Course/Deep Learning/archive", testPath)


trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

trainY=trainY.reshape(-1,)
testY=testY.reshape(-1,)

"""one-hot encode the training and testing labels
numLabels = len(np.unique(trainY))
trainY = to_categorical(trainY, numLabels)
testY = to_categorical(testY, numLabels)"""


# calculate the total number of images in each class and
# initialize a dictionary to store the class weights
classTotals = trainY.sum(axis=0)
classWeight = dict()


TypeError: object of type 'numpy.int32' has no len()

In [42]:
for i in range(0, classTotals.shape[0]):
    classWeight[i] = classTotals.max() / classTotals[i]

IndexError: tuple index out of range

In [100]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dense(43, activation='softmax')
])


In [101]:
cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [102]:
cnn.fit(trainX, trainY, epochs=11)


Epoch 1/11
1226/1226 [==============================] - 14s 11ms/step - loss: 3.5003 - accuracy: 0.0527
Epoch 2/11
1226/1226 [==============================] - 14s 11ms/step - loss: 3.2414 - accuracy: 0.1316
Epoch 3/11
1226/1226 [==============================] - 15s 12ms/step - loss: 1.2636 - accuracy: 0.6335
Epoch 4/11
1226/1226 [==============================] - 16s 13ms/step - loss: 0.5787 - accuracy: 0.8284
Epoch 5/11
1226/1226 [==============================] - 16s 13ms/step - loss: 0.3944 - accuracy: 0.8851
Epoch 6/11
1226/1226 [==============================] - 16s 13ms/step - loss: 0.3016 - accuracy: 0.9127
Epoch 7/11
1226/1226 [==============================] - 16s 13ms/step - loss: 0.2423 - accuracy: 0.9316
Epoch 8/11
1226/1226 [==============================] - 17s 13ms/step - loss: 0.2037 - accuracy: 0.9421
Epoch 9/11
1226/1226 [==============================] - 17s 14ms/step - loss: 0.1769 - accuracy: 0.9486
Epoch 10/11
1226/1226 [==============================] - 17s 14m

In [103]:
cnn.evaluate(testX,testY)


395/395 [==============================] - 2s 5ms/step - loss: 0.4224 - accuracy: 0.9000


[0.4223977029323578, 0.8999999761581421]

In [62]:
y_pred = cnn.predict(testX)
y_classes = [np.argmax(element) for element in y_pred]
y_classes[:5]

[3, 13, 26, 4, 8]

In [1]:
plot_sample(trainX*255, trainY*255, 80)

NameError: name 'plot_sample' is not defined